In [ ]:
# Code taken from https://realpython.com/k-means-clustering-python/#how-to-perform-k-means-clustering-in-python
import utils
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import \
    silhouette_score, adjusted_rand_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import numpy as np

df = utils.load_simple('digit-recognizer/train.csv')
df.head()

In [ ]:
features = df.loc[:, df.columns != 'label']
true_labels = df['label']

In [ ]:
# Standardize?
# scaler = StandardScaler()
# scaled_features = scaler.fit_transform(features)

In [ ]:
kmeans = KMeans(
    init = "k-means++",
    n_clusters = 10,
    n_init = 50,
    max_iter = 500,
    random_state = 42
)
kmeans.fit(features)

In [ ]:
predicted_labels = kmeans.labels_

print(f"Inertia: {kmeans.inertia_}")
print(f"Cluster Centres: {kmeans.cluster_centers_}")
print(f"Inertia: {kmeans.n_iter_}")
# A silhouette coefficient of 0 indicates that clusters are significantly overlapping one another, 
# and a silhouette coefficient of 1 indicates clusters are well-separated.
print(f"Silhouette Score: {silhouette_score(features, predicted_labels)}")
# An ARI score of 0 indicates that cluster labels are randomly assigned, 
# and an ARI score of 1 means that the true labels and predicted labels form identical clusters.
print(f"Inertia: {adjusted_rand_score(true_labels, predicted_labels)}")

In [ ]:
# Code taken from https://medium.com/@joel_34096/k-means-clustering-for-image-classification-a648f28bdc47
def retrieve_info(cluster_labels,y_train):
    """Associates most probable label with each cluster in KMeans model 
    returns: dictionary of clusters assigned to each label"""
    # Initializing
    reference_labels = {}
    # For loop to run through each label of cluster label
    for i in range(len(np.unique(kmeans.labels_))):
        index = np.where(cluster_labels == i,1,0)
        num = np.bincount(y_train[index==1]).argmax()
        reference_labels[i] = num
    return reference_labels

In [ ]:
reference_labels = retrieve_info(predicted_labels, true_labels)
number_labels = np.random.rand(len(predicted_labels))
for i in range(len(predicted_labels)):
    number_labels[i] = reference_labels[predicted_labels[i]]
print(reference_labels)

In [ ]:
confusion_matrix = metrics.confusion_matrix(true_labels, number_labels)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix)
cm_display.plot()
plt.show()

In [ ]:
print(true_labels[:20].tolist())
print(number_labels[:20].astype('int').tolist())

In [ ]:
# macro average does not take label imbalance into account
# micro and weighted averages return same value as accuracy score
print(f"Accuracy: {round(accuracy_score(true_labels, number_labels), 4) * 100}%")
print(f"Precision: {round(precision_score(true_labels, number_labels, average = 'macro'), 4) * 100}%")
print(f"Recall: {round(recall_score(true_labels, number_labels, average = 'macro'), 4) * 100}%")
print(f"F1 Score: {round(f1_score(true_labels, number_labels, average = 'macro'), 4) * 100}%")